# Exercício

---
- Treine um modelo de classificação DecisionTreeClassifier do pacote scikit-learn para classificar os produtos em suas categorias.
- Experimente a lib SpaCy para remover as stop words e reduzir as palavras a seu lema. Veja como essas alterações impactam o desempenho do classificador.


In [12]:
import pandas as pd

df = pd.read_csv(
    "https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8'
  ).sample(frac=0.5, random_state=42)

df.dropna(inplace=True)
df["texto"] = df['nome'] + " " + df['descricao']
df.head()

,nome,descricao,categoria,texto
33,Extraordinário,Produto Novo“Extraordinário” é um livro que co...,livro,Extraordinário Produto Novo“Extraordinário” ...
3316,Fifa 2018 Narração Português Completo Midia D...,FIFA 2018 - XBOX 360 MIDIA DIGITAL COMPLETOATE...,game,Fifa 2018 Narração Português Completo Midia D...
1557,Kit Mega Sarutobi Kunai Naruto Asuma Shuriken...,Descrição do Kit Ninja (Foto):Uma Kunai Asuma ...,brinquedo,Kit Mega Sarutobi Kunai Naruto Asuma Shuriken...
2548,Caixa 50 Cores Batom Queen Fosco Matte Nude K...,CAIXA C/ 50 CORES DIFERENTES!BATOM MATTE QUEEN...,maquiagem,Caixa 50 Cores Batom Queen Fosco Matte Nude K...
457,Box Dourado Crónica De Gelo E Fogo Edição De ...,A série Crônicas de Gelo e apresentações. São ...,livro,Box Dourado Crónica De Gelo E Fogo Edição De ...


In [2]:
df.categoria.value_counts()

livro        411
maquiagem    387
brinquedo    328
game         298
Name: categoria, dtype: int64

In [3]:
# Exemplo 1: Vetorização por contagem de termos simples com unigrama, sem stopwords do NLTK e modelo de classificação DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import nltk

# stopwords NLTK
nltk.download('stopwords')
stops = nltk.corpus.stopwords.words('portuguese')

# vetorização por contagem de termos
#vect = CountVectorizer(ngram_range=(1,1)) # exemplo 1.1: vetorização unigrama com stopwords
vect = CountVectorizer(ngram_range=(1,1), stop_words=stops) # exemplo 1.2: vetorização unigrama sem stopwords
vect.fit(df.texto)
text_vect = vect.transform(df.texto)

# divisão da amostra entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(
      text_vect, 
      df["categoria"], 
      test_size = 0.2, 
      random_state = 42
  )

# treinamento do modelo ávore de decisão
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

# escoragem da classificação na amostra de teste
y_prediction = tree.predict(X_test)

# mensuração do resultado pela acurácia
accuracy = accuracy_score(y_prediction, y_test)

print(text_vect.shape)
print(accuracy)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jeffe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
(1424, 23418)
0.9543859649122807


In [7]:
#!pip install spacy
#!python -m spacy download pt
#!python -m spacy download en

In [8]:
# função de lematização completa do documento
def lemmatizer_text(text):        
  sent = []
  doc = nlp(text)
  for word in doc:
      sent.append(word.lemma_)
  return " ".join(sent)

# função de lematização para os verbos do documento
def lemmatizer_verbs(text):        
  sent = []
  doc = nlp(text)
  for word in doc:
      if word.pos_ == "VERB":
          sent.append(word.lemma_)
      else:
          sent.append(word.orth_)
  return " ".join(sent)

In [10]:
# teste das funções de lematização
import spacy
nlp = spacy.load('pt_core_news_sm')

print(lemmatizer_text('correndo 1, 2, 3'))
print(lemmatizer_verbs('correndo 1, 2, 3'))

correr 1 , 2 , 3
correr 1 , 2 , 3


In [13]:
# aplica a lematização no dataframe
df['text_lemma'] = df.texto.apply(lemmatizer_text)
df['text_lemma_verbs'] = df.texto.apply(lemmatizer_verbs)

In [14]:
print(df.info())
print('\nshape: ', df.shape)
print(df.head(2))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1424 entries, 33 to 2615
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   nome              1424 non-null   object
 1   descricao         1424 non-null   object
 2   categoria         1424 non-null   object
 3   texto             1424 non-null   object
 4   text_lemma        1424 non-null   object
 5   text_lemma_verbs  1424 non-null   object
dtypes: object(6)
memory usage: 77.9+ KB
None

shape:  (1424, 6)
                                                   nome  \
33                                      Extraordinário    
3316   Fifa 2018 Narração Português Completo Midia D...   

                                              descricao categoria  \
33    Produto Novo“Extraordinário” é um livro que co...     livro   
3316  FIFA 2018 - XBOX 360 MIDIA DIGITAL COMPLETOATE...      game   

                                                  texto  \
33     Extraord

In [15]:
print(df['texto'][0])
print(df['text_lemma'][0])
print(df['text_lemma_verbs'][0])

O Hobbit - 7ª Ed. 2013  Produto NovoBilbo Bolseiro é um hobbit que leva uma vida confortável e sem ambições. Mas seu contentamento é perturbado quando Gandalf, o mago, e uma companhia de anões batem à sua porta e levam-no para uma expedição. Eles têm um plano para roubar o tesouro guardado por Smaug, o Magnífico, um grande e perigoso dragão. Bilbo reluta muito em participar da aventura, mas acaba surpreendendo até a si mesmo com sua esperteza e sua habilidade como ladrão!CaracterísticasAutor: Tolkien, J. R. R.Peso: 0.44I.S.B.N.: 9788578277109Altura: 20.000000Largura: 13.000000Profundidade: 1.000000Número de Páginas: 328Idioma: PortuguêsAcabamento: BrochuraNúmero da edição: 7Ano da edição: 2013
  O Hobbit - 7ª Ed . 2013   Produto NovoBilbo Bolseiro ser um hobbit que levar umar vidar confortável e sem ambição . Mas seu contentamento ser perturbar quando Gandalf , o mago , e umar companhia de anão bater à suar portar e levam-no parir umar expedição . Eles ter um planar parir roubar o teso

In [16]:
# Exemplo 2: Vetorização por contagem de termos simples com a combinação de unigrama e bigrama no documento lematizado, sem stopwords do SpaCy e modelo de classificação DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
import spacy

# stopwords SpaCy
nlp = spacy.load('pt_core_news_sm')
stops = nlp.Defaults.stop_words

# vetorização por contagem de termos no documento lematizado
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops) # exemplo 2.1: vetorização e combinação de unigrama e bigrama sem stopwords
vect.fit(df.text_lemma)
text_vect = vect.transform(df.text_lemma)

X_train,X_test,y_train,y_test = train_test_split(
    text_vect, 
    df["categoria"], 
    test_size = 0.2,
    random_state = 42
  )

tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

y_prediction = tree.predict(X_test)

accuracy = accuracy_score(y_prediction, y_test)

print(text_vect.shape)
print(accuracy)

OSError: [E050] Can't find model 'pt'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [ ]:
# Exemplo 3: Vetorização por contagem de termos simples com a combinação de unigrama e bigrama no documento com verbos lematizado, sem stopwords do SpaCy e modelo de classificação DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import spacy

# stopwords SpaCy
nlp = spacy.load('pt')
stops = nlp.Defaults.stop_words

# vetorização por contagem de termos no documento com os verbos lematizado
#vect = CountVectorizer(ngram_range=(1,1), stop_words=stops) # exemplo 3.1: vetorização e combinação de unigrama sem stopwords
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops) # exemplo 3.2: vetorização e combinação de unigrama e bigrama sem stopwords
vect.fit(df.text_lemma_verbs)
text_vect = vect.transform(df.text_lemma_verbs)

X_train,X_test,y_train,y_test = train_test_split(
    text_vect, 
    df["categoria"], 
    test_size = 0.2,
    random_state = 42
  )

tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

y_prediction = tree.predict(X_test)

accuracy = accuracy_score(y_prediction, y_test)

print(text_vect.shape)
print(accuracy)

In [ ]:
# Exemplo 4: Vetorização por contagem de termos simples com a combinação de unigrama e bigrama no documento com verbos lematizado, sem stopwords do SpaCy e NLTK combinadas e modelo de classificação DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import spacy
import nltk

nltk.download('stopwords')
nlp = spacy.load('pt')

# stopwords do SpaCy e NLTK combinadas
stops = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))
#len(stops)

# vetorização por contagem de termos no documento com os verbos lematizado
vect = CountVectorizer(ngram_range=(1,2), stop_words=stops)
vect.fit(df.text_lemma_verbs)
text_vect = vect.transform(df.text_lemma_verbs)

X_train,X_test,y_train,y_test = train_test_split(
    text_vect, 
    df["categoria"], 
    test_size = 0.2,
    random_state = 42
  )

tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

y_prediction = tree.predict(X_test)

accuracy = accuracy_score(y_prediction, y_test)

print(text_vect.shape)
print(accuracy)

In [ ]:
# Exemplo 5: Vetorização por contagem de termos simples com a combinação de unigrama com documentos lematizado, sem stopwords do SpaCy e NLTK combinadas e modelo de classificação DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import spacy
import nltk

nltk.download('stopwords')
nlp = spacy.load('pt')

# stopwords do SpaCy e NLTK combinadas
stops_spacy = nlp.Defaults.stop_words
stops_nltk = nltk.corpus.stopwords.words('portuguese')
stops = list(set(stops_spacy).union(set(stops_nltk)))

# vetorização por contagem de termos no documento lematizado
vetorTfidf = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops, norm='l2')
#vetorTfidf = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops_spacy, norm='l2')
#vetorTfidf = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stops_nltk, norm='l2')
vetorTfidf.fit(df.text_lemma)
text_vect = vetorTfidf.transform(df.text_lemma)

X_train,X_test,y_train,y_test = train_test_split(
    text_vect, 
    df["categoria"], 
    test_size = 0.2,
    random_state = 42
  )

tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

y_prediction = tree.predict(X_test)

accuracy = accuracy_score(y_prediction, y_test)

print(text_vect.shape)
print(accuracy)